In [19]:
# LunarLander debug note
# Initially, used the same code I used for cartpole.
# Would not train, so printed out the probs. When the model is first made, it is fairly distributed like this [[0.2660, 0.2257, 0.2731, 0.2352]]. However, after training for 1000 steps,
# it converges to a certain action like this - tensor([[9.9943e-01, 4.8898e-04, 7.9276e-05, 4.1656e-08]]
# Applied entropy for more exploration. Didn't work
# Found much larger absolute value of advantages, returns, compared to cartpole. Especially, when terminated, the return is -100 which is the dominant cause for the returns.
# Normalizing advantage solved the problem above. Now it doesn't fixate in a certain action.
# Then, action_probability turned into a somewhat uniform distribution. Therefore, printed out the losses.
# Actor loss:  tensor(-1.9073e-08, device='cuda:0', grad_fn=<NegBackward0>) Critic loss:  tensor(15.9172, device='cuda:0', grad_fn=<MseLossBackward0>)
# Entropy:  tensor(1.3784, device='cuda:0', grad_fn=<MeanBackward0>)
# Found out that Actor loss is extremely small. Therefore, actor layer was barely getting updated.
# Reason: log prob is pretty much uniform when model is initialized and the advantage is normalized with mean 0. We calculate actor_loss = (advantages * log_probs).mean()
# This converges to the mean of the normal distribution (since log_probs is uniform), and therefore to 0.
# Try with normalizing the n-step rewards instead of the advantage.
# Reference https://github.com/nikhilbarhate99/Actor-Critic-PyTorch/blob/master/train.py
# The only difference between my model and the reference model was the reduction method of actor and critic loss. Reference model used sum, my model used mean.
# mean vs. sum was not the problem. The dimensions of log_probs, returns, state_values were all different in my code and was broadcasting very wierdly. That's why step-wise calculation of the reference code
# worked, but my code didn't. Should pay more attention to dimensions, and also debugging warnings because I was ignoring the dimension disparity.
# Trains well using mean. Maybe even better.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [7]:
!pip install gymnasium
!pip install pygame
!pip install wheel setuptools pip --upgrade
!pip install swig
!pip install gymnasium[box2d]

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'box2d-py' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'box2d-py'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
ERROR: Failed to build installable wheels for some pyproject.toml based projects (box2d-py)


In [8]:
!pip uninstall -y box2d-py
!pip install box2d pygame swig
!pip install "gymnasium[box2d]" --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 102.3 MB/s eta 0:00:00


In [2]:
# New implementation following https://github.com/nikhilbarhate99/Actor-Critic-PyTorch/blob/master/train.py

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

class ActorCritic(nn.Module):

    # Increased hidden dim to (128, 128) compared to cartpole as the input_dim is more complex
    def __init__(self, input_dim, output_dim, hidden_dims=(128, 128)):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            # nn.Linear(hidden_dims[0], hidden_dims[1]),
            # nn.ReLU()
        )
        self.actor_layer = nn.Linear(hidden_dims[0], output_dim)
        self.critic_layer = nn.Linear(hidden_dims[0], 1)

        self.logprobs = []
        self.state_values = []
        self.rewards = []

    def forward(self, x):
        x = self.net(x)
        action_probs = F.softmax(self.actor_layer(x), dim=-1)
        state_value = self.critic_layer(x)

        action_distribution = Categorical(action_probs)
        action = action_distribution.sample()

        self.logprobs.append(action_distribution.log_prob(action))
        self.state_values.append(state_value.squeeze())

        return action.item()

    def compute_return(self, gamma):
        returns = []
        R = 0
        for r in reversed(self.rewards):
            R = r + gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        return returns

    def calculate_loss(self, gamma):
        rewards = self.compute_return(gamma).detach()

        # CRITICAL BUG - log_probs' shape was [90, 1], state_values' [90, 1, 1], rewards' [90] -- should read the warning messages more carefully from now on.
        # print("log_probs.shape", self.logprobs.shape)
        # print("state_values.shape", self.state_values.shape)
        # print("returns.shape", returns.shape)

        loss = 0
        for logprob, value, reward in zip(self.logprobs, self.state_values, rewards):
            value = value.to('cpu')
            advantage = reward - value.detach()
            action_loss = -logprob * advantage
            value_loss = F.smooth_l1_loss(value, reward)
            loss += (action_loss + value_loss)
        return loss


    def clearMemory(self):
        del self.logprobs[:]
        del self.state_values[:]
        del self.rewards[:]



In [3]:
from tqdm import tqdm

class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, num_steps = 5):
        random_seed = 543
        torch.manual_seed(random_seed)
        self.env = env
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.num_steps = num_steps
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = ActorCritic(env.observation_space.shape[0], env.action_space.n).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)

    def train(self):
        episode_rewards = []
        episode_steps = []

        for episode in tqdm(range(self.num_episodes)):
            state, _ = self.env.reset()
            episode_reward = 0
            steps = 0
            done = False

            while not done and steps < self.max_steps:
                steps += 1
                state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
                action = self.policy_net(state_tensor)

                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                self.policy_net.rewards.append(reward)
                episode_reward += reward
                state = next_state

                # every n steps, calculate losses, update the actor & critic, then refresh the saved lists
                # if (steps % self.num_steps == 0) or done:
                if done:
                    self.optimizer.zero_grad()
                    loss = self.policy_net.calculate_loss(self.gamma)
                    loss.backward()
                    self.optimizer.step()
                    self.policy_net.clearMemory()

            if episode % 100 == 0:
                print(episode, "reward: ", episode_reward, "steps: ", steps)

            episode_rewards.append(episode_reward)
            episode_steps.append(steps)

        self.env.close()
        return np.array(episode_rewards), np.array(episode_steps)


In [ ]:
import gymnasium as gym

env = gym.make('LunarLander-v3')
env.reset(seed=543)
num_episodes = 2000
max_steps = 1000
lr = 0.02


a2c_model_ll =  A2CAgent(env, num_episodes=num_episodes, max_steps=max_steps, lr=lr, num_steps = 8)

state, _ = env.reset()
state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(a2c_model_ll.device)
rewards, steps = a2c_model_ll.train()


In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(rewards)), rewards)
plt.show()

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class ActorCritic(nn.Module):

    # reference model used only 1 layer. Will experiment both
    def __init__(self, input_dim, output_dim, hidden_dims=(128, 128)):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            # nn.Linear(hidden_dims[0], hidden_dims[1]),
            # nn.ReLU()
        )
        self.actor_layer = nn.Linear(hidden_dims[1], output_dim)
        self.critic_layer = nn.Linear(hidden_dims[1], 1)

    def forward(self, x):
        x = self.net(x)
        action_probs = F.softmax(self.actor_layer(x), dim=-1)
        value = self.critic_layer(x)
        return action_probs, value


In [14]:
from tqdm import tqdm

class A2CAgent:
    def __init__(self, env_id, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, num_steps = 5, num_envs = 8, vectorization_mode = "sync"):
        # using vectorized environments to boost training speed
        self.env = gym.make_vec(env_id, num_envs = num_envs, vectorization_mode=vectorization_mode)
        self.env.reset(seed=543)
        self.num_envs = num_envs
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.num_steps = num_steps
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = ActorCritic(self.env.single_observation_space.shape[0], self.env.single_action_space.n).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        self.loss = nn.MSELoss()

    # choosing action from policy's probability distribution
    def choose_action(self, state):
        probs, _ = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action

    # computing the gamma decaying rewards
    def compute_returns(self, rewards):
        """
        Args:
            rewards: torch.Tensor of shape [T, N] where
                    T = rollout steps, N = num_envs
        Returns:
            returns: torch.Tensor of shape [T, N], normalized
        """
        rewards = torch.stack(rewards)

        T, N = rewards.shape
        returns = torch.zeros_like(rewards)
        R = torch.zeros(N, device=rewards.device)
        for t in reversed(range(T)):
            R = rewards[t] + self.gamma * R
            returns[t] = R

        # Normalize returns across all timesteps and environments
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)

        return returns


    # computing the n step rewards
    def compute_n_step_returns(self, rewards, next_value):
        # bootstraps the future reward using value estimate
        R = next_value
        returns = []
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.stack(returns)
        if returns.numel() > 1:
            return (returns - returns.mean()) / (returns.std() + 1e-8)
        else:
            return returns * 0

    def train(self):
        episode_rewards = []
        episode_steps = []

        for episode in tqdm(range(self.num_episodes)):
            state, _ = self.env.reset()
            done = np.zeros(self.num_envs, dtype=bool)
            episode_reward = np.zeros(self.num_envs)
            values, rewards, log_probs = [], [], []
            done_mask = np.zeros(self.num_envs, dtype=bool)
            done_steps = np.zeros(self.num_envs)
            steps = 0

            while not np.all(done_mask) and steps < self.max_steps:
                steps += 1
                state_tensor = torch.tensor(state, dtype=torch.float32).to(self.device)
                action_probs, value = self.policy_net(state_tensor)
                action_dist = torch.distributions.Categorical(action_probs)
                action = action_dist.sample()
                log_prob = action_dist.log_prob(action)

                next_state, reward, terminated, truncated, _ = self.env.step(action.cpu().numpy())
                done = np.logical_or(terminated, truncated)
                done_steps = np.where(np.logical_and(done, ~done_mask), steps, done_steps)
                done_mask = np.logical_or(done_mask, done)
                # record when each environment is done
                reward = np.where(done_mask, 0.0, reward)

                # saves the values, rewards, log_probs which are used to calculate the n_step returns, actor loss, and critic loss
                values.append(value.squeeze())
                rewards.append(torch.tensor(reward, dtype=torch.float32).to(self.device))  # shape: (num_envs,)
                log_probs.append(log_prob)

                episode_reward += reward
                state = next_state


                # every n steps for each environment, calculate losses, update the actor & critic, then refresh the saved lists
                # if (steps % self.num_steps == 0) or np.any(done):
                if np.any(done):
                    with torch.no_grad():
                        next_state_tensor = torch.tensor(next_state, dtype=torch.float32).to(self.device)
                        _, next_value = self.policy_net(next_state_tensor)
                        done_tensor = torch.tensor(done, dtype=torch.float32).to(self.device)
                        next_value = next_value.squeeze() * (1 - done_tensor)

                    returns = self.compute_returns(rewards)  # shape: (n_steps, num_envs)
                    returns = returns.transpose(0, 1)  # shape: (num_envs, n_steps)
                    values = torch.stack(values).transpose(0, 1)  # shape: (num_envs, n_steps)
                    log_probs = torch.stack(log_probs).transpose(0, 1)  # shape: (num_envs, n_steps)
                    advantages = returns - values

                    # calculate sum instead of mean
                    actor_loss = - (log_probs * advantages.detach()).sum()
                    critic_loss = self.loss(returns, values)

                    loss = actor_loss + critic_loss
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                    values = []
                    rewards = []
                    log_probs = []

            episode_rewards.append(episode_reward)
            episode_steps.append(steps)

            if episode % 20 == 0:
               print('Episode {}\tlengths: {}\treward: {}]\tfull length: {}'.format(episode, done_steps, episode_reward, steps))

            episode_rewards.append(episode_reward)
            episode_steps.append(steps)

        self.env.close()
        return np.array(episode_rewards), np.array(episode_steps)


In [12]:
import gymnasium as gym

env_id = 'LunarLander-v3'
num_episodes = 1000
max_steps = 1000
lr = 0.02


a2c_model_ll =  A2CAgent(env_id, num_episodes=num_episodes, max_steps=max_steps, lr=lr, num_steps = 8)

rewards, steps = a2c_model_ll.train()



  0%|          | 1/1000 [00:01<27:29,  1.65s/it]

Episode 0	lengths: [115. 112. 192.  92. 118.  60. 123.  84.]	reward: [-245.34536021 -191.66144945 -500.03720977  -11.33437027 -178.22374885
    9.63657237 -559.72998484   96.26161105]]	full length


  2%|▏         | 21/1000 [00:23<07:55,  2.06it/s]

Episode 20	lengths: [101. 167.  94. 102.  70.  97.  95.  69.]	reward: [-155.8002251  -243.81072087 -426.3456884    24.2093625  -445.11672289
 -281.91675918  -47.03076698 -378.58775796]]	full length


  4%|▍         | 41/1000 [00:42<29:40,  1.86s/it]

...

 96%|█████████▌| 961/1000 [28:06<00:44,  1.15s/it]

Episode 960	lengths: [189. 188. 245. 240. 178. 337. 126. 201.]	reward: [136.20201953  73.94018374  96.70209153  88.408784   -52.37408652
 172.53432012 -43.3308698  -89.94665456]]	full length


 98%|█████████▊| 981/1000 [28:26<00:19,  1.02s/it]

Episode 980	lengths: [145.  80. 214. 241. 188. 378. 382.  85.]	reward: [-43.64471542  35.81622811  86.844281    99.04345521 -62.70261642
 155.28235398  51.19856435 101.08168946]]	full length


100%|██████████| 1000/1000 [28:48<00:00,  1.73s/it]


In [17]:
env_id = 'LunarLander-v3'
num_episodes = 1000
max_steps = 1000
lr = 0.02

a2c_model_ll_async =  A2CAgent(env_id, num_episodes=num_episodes, max_steps=max_steps, lr=lr, num_steps = 8, vectorization_mode="async")

rewards_async, steps_async = a2c_model_ll_async.train()

  0%|          | 1/1000 [00:00<08:47,  1.89it/s]

Episode 0	lengths: [109. 105.  91.  86. 108.  71. 120.  88.]	reward: [-384.05631434 -244.53897027 -475.32608407 -237.94813173 -252.59032813
 -274.03278797 -330.4445024  -269.70100465]]	full length: 120


  2%|▏         | 21/1000 [01:02<55:04,  3.38s/it]

Episode 20	lengths: [ 196. 1000.  796.  218. 1000.  159. 1000. 1000.]	reward: [  82.64851266   36.39450977   -7.41036394  -48.31003876 -148.97778193
  -32.81208858 -198.13951374    6.12669533]]	full length: 1000


  4%|▍         | 41/1000 [01:22<13:59,  1.14it/s]

...

 96%|█████████▌| 961/1000 [49:40<02:28,  3.80s/it]

Episode 960	lengths: [ 155.  363. 1000.  468.  148.  355.  116.  451.]	reward: [ -21.17706031  -37.88632335 -125.1789049   -69.86483816  -27.39609436
  -88.35731558  -48.01772539  -36.389359  ]]	full length: 1000


 98%|█████████▊| 981/1000 [50:58<01:17,  4.07s/it]

Episode 980	lengths: [ 322.  219.  565. 1000.  831. 1000.  728. 1000.]	reward: [ -55.8004386   -20.76729692  -68.17674496 -125.28326551 -116.31863609
  -99.77122249 -108.74760863  -98.86020141]]	full length: 1000


100%|██████████| 1000/1000 [52:13<00:00,  3.13s/it]


In [21]:
import gymnasium as gym
import torch
import numpy as np
from gymnasium.wrappers import RecordVideo
import os

# Create folder to save the video
video_folder = "./video"
os.makedirs(video_folder, exist_ok=True)

# Wrap the environment with RecordVideo
env = gym.make('LunarLander-v3', render_mode='rgb_array')
env = RecordVideo(env, video_folder=video_folder, episode_trigger=lambda e: True)

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

episode_rewards = []

for i in range(num_episodes):
    state, _ = env.reset()
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
    episode_reward = 0
    done = False

    while not done:
        with torch.no_grad():
            action_probs, _ = a2c_model_ll.policy_net(state)
            # action_dist = torch.distributions.Categorical(action_probs)
            # action = action_dist.sample().item()
            action = torch.argmax(action_probs, dim=1).item()

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward

        state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0).to(device)

    episode_rewards.append(episode_reward)
    print(f"Episode {i+1} Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")


Episode 1 Reward: -52.894699397403144
Episode 2 Reward: -27.627745976441886
Episode 3 Reward: -236.95609091731924
Episode 4 Reward: 135.61399384725155
Episode 5 Reward: -66.2182336378679
Episode 6 Reward: 19.699363217334394
Episode 7 Reward: 26.81668761972692
Episode 8 Reward: -15.518179226312213
Episode 9 Reward: 13.410176329457428
Episode 10 Reward: -11.25524196136928
Average Reward over 10 episodes: -21.49299701029434


In [22]:
import gymnasium as gym
import torch
import numpy as np
from gymnasium.wrappers import RecordVideo
import os

# Create folder to save the video
video_folder = "./video"
os.makedirs(video_folder, exist_ok=True)

# Wrap the environment with RecordVideo
env = gym.make('LunarLander-v3', render_mode='rgb_array')
env = RecordVideo(env, video_folder=video_folder, name_prefix="async_demo", episode_trigger=lambda e: True)

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

episode_rewards = []

for i in range(num_episodes):
    state, _ = env.reset()
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
    episode_reward = 0
    done = False

    while not done:
        with torch.no_grad():
            action_probs, _ = a2c_model_ll_async.policy_net(state)
            # action_dist = torch.distributions.Categorical(action_probs)
            # action = action_dist.sample().item()
            action = torch.argmax(action_probs, dim=1).item()

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward

        state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0).to(device)

    episode_rewards.append(episode_reward)
    print(f"Episode {i+1} Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")


Episode 1 Reward: -113.4247666969921
Episode 2 Reward: -136.1491438008048
Episode 3 Reward: -113.35021589916234
Episode 4 Reward: -108.9208804584625
Episode 5 Reward: -88.38824172533324
Episode 6 Reward: -127.98817557744319
Episode 7 Reward: -129.28597991379633
Episode 8 Reward: -117.62409761534742
Episode 9 Reward: -104.91873832786548
Episode 10 Reward: -92.15144943763181
Average Reward over 10 episodes: -113.2201689452839
